In [1]:
import numpy as np

In [1]:
def bresenham3D(x1, y1, z1, x2, y2, z2):
    points = []

    # Calculate differences and absolute differences
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)

    # Determine the sign of the differences
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    sz = 1 if z1 < z2 else -1

    # Initialize error variables
    err1 = dx - dy
    err2 = dx - dz

    # Set the starting point
    x, y, z = x1, y1, z1

    while x != x2 or y != y2 or z != z2:
        # Add the current point to the list of points
        points.append((x, y, z))

        print(points)

        # Calculate error terms
        e1 = err1 * 2
        e2 = err2 * 2

        # Adjust the error terms based on the step directions
        if e1 > -dy:
            err1 -= dy
            x += sx
        if e1 < dx:
            err1 += dx
            y += sy

        if e2 > -dz:
            err2 -= dz
            x += sx
        if e2 < dx:
            err2 += dx
            z += sz

    # Add the last point to the list
    points.append((x2, y2, z2))

    return points

# Example usage:
point1 = (0, 0, 0)
point2 = (4, 2, 3)

result = bresenham3D(*point1, *point2)
print(result)


KeyboardInterrupt: 

In [2]:
def euclidean_distance(point1, point2):
    """
    Calculate the Euclidean distance between two 3D points.

    Parameters:
    point1 (tuple): The first 3D point (x, y, z).
    point2 (tuple): The second 3D point (x, y, z).

    Returns:
    float: The Euclidean distance between point1 and point2.
    """
    return np.linalg.norm(np.array(point1) - np.array(point2))

In [4]:
def calculate_single_path_fitness(x, ps, pt, constant_speed, energy_weight):
    """
    Calculate the total distance and energy consumption for a single path x.

    Parameters:
    x (list of tuples): A list of control points defining the path.
    ps (tuple): The start point (x, y, z).
    pt (tuple): The target point (x, y, z).
    constant_speed (float): The constant speed of the drone.
    energy_weight (float): The weight of energy consumption in the fitness function.

    Returns:
        float: The fitness value for the path.
    """
    num_points = len(x)
    if num_points < 2:
        return 0.0  # No movement if there are no control points

    total_distance = euclidean_distance(ps, x[0])  # Distance from start to the first control point

    # Distance between all control points
    for i in range(1, num_points - 1):
        total_distance += euclidean_distance(x[i], x[i + 1])

    total_distance += euclidean_distance(x[-1], pt)  # Distance from the last control point to the target


    # Calculate energy consumption based on distance and constant speed
    energy_consumed = total_distance / constant_speed

    # Calculate the fitness value as a weighted sum of time and energy
    fitness = total_distance + energy_weight * energy_consumed

    return fitness



In [12]:
import numpy as np
elite_indices = np.argsort([3,5,2,4])
print(elite_indices[-2:])

[1]


In [7]:

def calculate_total_fitness(drone_paths, ps_list, pt_list, constant_speed, energy_weight):
    """
    Calculate the total distance for multiple drones' paths.

    Parameters:
    drone_paths (list of lists): A list of paths, each defined as a list of control points.
    ps_list (list of tuples): A list of start points for each drone.
    pt_list (list of tuples): A list of target points for each drone.
    constant_speed (float): The constant speed of the drone.
    energy_weight (float): The weight of energy consumption in the fitness function.

    Returns:
    float: The total fitness value traveled along all paths from ps to pt.
    """
    total_fitness = 0.0

    for i in range(len(drone_paths)):
        path = drone_paths[i]
        ps = ps_list[i]
        pt = pt_list[i]
        fitness = calculate_single_path_fitness(path, ps, pt, constant_speed, energy_weight)
        total_fitness += fitness

    return total_fitness



In [ ]:
def check_energy_constraint(ps, pt, drone_paths, constant_speed, maximum_energy):
    """
    Check if the total energy consumed by all drones satisfies the energy constraint.

    Args:
        drone_paths (list): List of drone paths where each path is a list of 3D points.
        constant_speed (float): Constant speed of the drones.
        maximum_energy (float): Maximum allowable total energy consumption.

    Returns:
        bool: True if the energy constraint is satisfied, False otherwise.
    """
    total_energy = 0.0
    for path, psi, pti in zip(drone_paths, ps, pt):
        total_distance = calculate_single_path_fitness(path, psi, pti, constant_speed, 0)
        energy_consumed = total_distance / constant_speed
        total_energy += energy_consumed

    if total_energy <= maximum_energy:
        return True  # Energy constraint is satisfied
    else:
        return False  # Energy constraint is violated

In [9]:
def check_collision_constraint(drone_paths, obstacle_list):
    """
    Check if drone paths collide with each other or with obstacles.

    Args:
        drone_paths (list): List of drone paths where each path is a list of 3D points.
        obstacle_list (list): List of obstacle representations.

    Returns:
        bool: True if the collision constraint is satisfied, False otherwise.
    """
    for i in range(len(drone_paths)):
        for j in range(i + 1, len(drone_paths)):
            path1 = drone_paths[i]
            path2 = drone_paths[j]
            for point1 in path1:
                for point2 in path2:
                    if euclidean_distance(point1, point2) < minimum_collision_distance:
                        return False  # Collisions detected

    for path in drone_paths:
        for point in path:
            for obstacle in obstacle_list:
                if euclidean_distance(point, obstacle) < minimum_collision_distance:
                    return False  # Collisions with obstacles detected

    return True  # No collisions with other drones or obstacles


In [8]:
# Example usage:
# Define paths for three drones
drone1_path = [(0, 0, 0), (1, 0, 0), (1, 1, 0), (2, 1, 0)]
drone2_path = [(0, 0, 0), (0, 1, 0), (1, 1, 0), (1, 2, 0)]
drone3_path = [(0, 0, 0), (1, 0, 0), (1, 1, 0), (2, 1, 0)]

# Define start and target points for the drones
ps_list = [(0, 0, 0), (0, 0, 0), (0, 0, 0)]
pt_list = [(2, 2, 0), (2, 2, 0), (2, 2, 0)]

# Constant speed of drones (e.g., 1.0 units per time step)
constant_speed = 1.0

# Weight for energy consumption in the fitness function
energy_weight = 0.1

# Create a list of drone paths
drone_paths = [drone1_path, drone2_path, drone3_path]

# Calculate the total fitness for all drones
total_fitness = calculate_total_fitness(drone_paths, ps_list, pt_list, constant_speed, energy_weight)
print("Total Fitness for All Drones:", total_fitness)

Total Fitness for All Drones: 9.899999999999999
